# Chapter 5-Domain Modelling with Types

Now I now we've just done a load of types. But we're gonna be doing more.

The idea of this chapter is what if we could use soruce code as documentation? Sounds great in theory, but bad in practice? Types can do that! (the book is very optimistic!)

### Simple Values

So when making a domain model we want to break down structures and entities into building blocks. We'll start with simple values.
Here's a basic example of a simple entity:

In [2]:
type CustomerId = CustomerId of int

Error: (1,30): error CS1002: ; expected
(1,33): error CS1002: ; expected
(1,36): error CS1519: Invalid token '' in class, record, struct, or interface member declaration

In [3]:
//We'limitedHashl create the type:
type CustomerId = CustomerId of int

//Then use it as a constructor:
let customerId = CustomerId 42

//These wrapper types are very useful as we can use them to compare equality:
type OrderId = OrderId of int
let orderId = OrderId 42

printfn "%b" (orderId = customerId)

Error: (2,30): error CS1002: ; expected
(2,33): error CS1002: ; expected
(5,5): error CS1002: ; expected
(5,29): error CS1002: ; expected
(5,31): error CS1002: ; expected
(8,24): error CS1002: ; expected
(8,27): error CS1002: ; expected
(9,5): error CS1002: ; expected
(9,23): error CS1002: ; expected
(9,25): error CS1002: ; expected
(11,9): error CS1002: ; expected

Very nice! We can keep simple values seperate even if they're the same primative value! 
This works for function definitions as well, so wrappers are very useful to keep things organised.

Most of the time, we want simple values to be bound by some constraints. We can do that! But it's in the next chapter.

#### A note on performance:
Most of the time this isn't an issue but we use big sets of data. Rather than defining a collection of wrappers, define a collecction with a wrapper:
- type ListOfThings = ListOfThings of int[]

## Complex Data

So simple values are funa nd all, but usually we want to use more complex models. For instance, something close to my heart, we would liek a CoP record or something made up of simple and complex values. This is very easy though! Say we wanted to map out a basic order structure:

In [4]:
type Order = {
    CustomerInfo : CustomerInfo
    ShippingAddress : ShippingAddress
    BillingAddress : BillingAddress 
    OrderLines : OrderLine list
    OrderQuantity : OrderQuantity
}
//This is essentially the equivalent of saying Order = customerInfo AND ShippingAddress AND BillingAddress AND ...

Error: (2,18): error CS1003: Syntax error, ',' expected
(2,20): error CS1003: Syntax error, ',' expected
(2,32): error CS1003: Syntax error, ',' expected
(3,21): error CS1003: Syntax error, ',' expected
(3,23): error CS1003: Syntax error, ',' expected
(3,38): error CS1003: Syntax error, ',' expected
(4,20): error CS1003: Syntax error, ',' expected
(4,22): error CS1003: Syntax error, ',' expected
(4,36): error CS1003: Syntax error, ',' expected
(5,16): error CS1003: Syntax error, ',' expected
(5,18): error CS1003: Syntax error, ',' expected
(5,28): error CS1003: Syntax error, ',' expected
(5,32): error CS1003: Syntax error, ',' expected
(6,19): error CS1003: Syntax error, ',' expected
(6,21): error CS1003: Syntax error, ',' expected
(7,2): error CS1002: ; expected

There's still a bit of work to do here though- none of these types have been defined yet, and these should be defined early on when modelling a domain. If we did want to use temporary values though, we can define an undefined type: 
- type Undefined = exn

This is actually just creating an alias for the F# exception type, which is great because if we try and actually use Undefined we'll get an error which (hopefully) reminds us to actually assign a type.

### Choice Types
Let's take a closer look at OrderQuantity. Depending on the type of order, we may want to use weight or the amount of objects (e.g. 5 books, 150g flour, etc.) but we don't really want to change the type OrderQuantity because that still makes sense. We can use choice types! Lucas covered this a lot last time but here's an example:

In [5]:
type UnitQuantity = UnitQuantity of int
type KilogramQuantity = KilogramQuantity of double

type OrderQuantity =
    | Unit of UnitQuantity
    | Kilogram of KilogramQuantity

//Very nice indeed

Error: (1,34): error CS1002: ; expected
(1,37): error CS1002: ; expected
(2,6): error CS1002: ; expected
(2,42): error CS1002: ; expected
(2,45): error CS1002: ; expected
(4,6): error CS1002: ; expected
(4,21): error CS1525: Invalid expression term '|'
(5,12): error CS1002: ; expected
(5,27): error CS1002: ; expected
(5,27): error CS1525: Invalid expression term '|'
(6,16): error CS1002: ; expected
(6,35): error CS1002: ; expected

### Workflows as Functions
Now we've defined structures, we want to do stuff with them. For this we we will obviosuly use functions.

So. In F# the functions we use only take one input and one output (sort of- we can use parameters in the input but we'll come back to that). But often we want to have multiple _possible_ outputs to a function- how do we do that?
Rather simply, create an output type:

In [6]:
//Let's assume we want to open an evelope that can be either an order or a quote and catagorize the contents, i.e. we want to take an EnvelopeContents type:
type EnvelopeContents = EnvelopeContents of string

type QuoteForm = QuoteForm of string
type OrderForm = OrderForm of string
//And convert it to a categorized form. This form can be different depending on the contents of the evelope, so we have to define our CategorizedMail type differently:
type CategorizedMail = 
    | Quote of QuoteForm
    | Order of OrderForm

//Then we can define the function type like so:
type CategorizeInboundMail = EnvelopeContents -> CategorizedMail

Error: (2,42): error CS1002: ; expected
(2,45): error CS1002: ; expected
(4,6): error CS1002: ; expected
(4,28): error CS1002: ; expected
(4,31): error CS1002: ; expected
(5,6): error CS1002: ; expected
(5,28): error CS1002: ; expected
(5,31): error CS1002: ; expected
(7,6): error CS1002: ; expected
(7,23): error CS1525: Invalid expression term '|'
(8,13): error CS1002: ; expected
(8,25): error CS1002: ; expected
(8,25): error CS1525: Invalid expression term '|'
(9,13): error CS1002: ; expected
(9,25): error CS1002: ; expected
(12,65): error CS1002: ; expected

Thus we can use CategoriseInboundMail and know which types will be converted to what. It's pretty cool!

We can do the same sort of thing for inputs as well- say we had a function CalculatePrices that takes in an OrderForm and a PriceCatalogue. We _could_ define the input type like so:

In [7]:
type CalculatePriceInput = {
    OrderForm : OrderForm
    PriceCatalogue : PriceCatalogue
}

//This means the function type definition would look like this:
type CalculatePrices = CalculatePriceInput -> Price

Error: (2,15): error CS1003: Syntax error, ',' expected
(2,17): error CS1003: Syntax error, ',' expected
(2,26): error CS1003: Syntax error, ',' expected
(3,20): error CS1003: Syntax error, ',' expected
(3,22): error CS1003: Syntax error, ',' expected
(4,2): error CS1002: ; expected
(7,52): error CS1002: ; expected

HOWEVER. This might not be the best approach. If the inputs are more dependencies than real inputs (like PriceCatalogue in the previous example) we probably just want to use it as a seperate parameter. Thus the function definition looks more like this:

In [8]:
type CalculatePrices = OrderForm -> PriceCatalogue -> Price

Error: (1,60): error CS1002: ; expected

I personally prefer the input structure way of doing things but this isn't really about preference- if the parameters are all strongly connected and always required, it's best to put them in a structure. Otherwise, parameters!


This is a touch theoretical however. In reality, we're probably going to get errors and want to know what errors and where we are going to get them. We can use the Result type for this that was covered in the previous chapter:
- Result<Price, PriceError list>

Very simple, we simply define the type and the error we expect within the <>

### Identity- Who Am I?

When dealing with data types and structures, often we want to be able to identify them. If we need a special ID to do this (i.e. the instance of the strcutre is unique) this is called an **entity**. Otherwise it's just a **value object**.

Value objects are easy enough to deal with- when equating them to other value objects, they are considered equal if all fields match (int he case of a choice value, it has to have the same choice case). For those of you who are on the ball, you'll know this is **structural equality**.

Entities are slightly trickier, but not much. For these we need an Id field of some kind:

In [9]:
type Contact = {
    ContactId : ContactId
    PhoneNumber : PhoneNumber
    EmailAddress : EmailAddress
}

Error: (2,15): error CS1003: Syntax error, ',' expected
(2,17): error CS1003: Syntax error, ',' expected
(2,26): error CS1003: Syntax error, ',' expected
(3,17): error CS1003: Syntax error, ',' expected
(3,19): error CS1003: Syntax error, ',' expected
(3,30): error CS1003: Syntax error, ',' expected
(4,18): error CS1003: Syntax error, ',' expected
(4,20): error CS1003: Syntax error, ',' expected
(5,2): error CS1002: ; expected

Very simple, but what if we have a choice field? Do we put the id on the top level or within each choice? Well, there's an easy answer to that- it's usally best to put it in the choice. Let me show you what I mean:

In [10]:
type Choice1 = {
    id : id
    value : value
    }
type Choice2 = {
    id : id
    value : value
    }

type Which = 
    | Choice1 of Choice1
    | Choice2 of Choice2


Error: (2,8): error CS1003: Syntax error, ',' expected
(2,10): error CS1003: Syntax error, ',' expected
(2,12): error CS1003: Syntax error, ',' expected
(3,11): error CS1003: Syntax error, ',' expected
(3,13): error CS1003: Syntax error, ',' expected
(4,6): error CS1002: ; expected
(6,8): error CS1003: Syntax error, ',' expected
(6,10): error CS1003: Syntax error, ',' expected
(6,12): error CS1003: Syntax error, ',' expected
(7,11): error CS1003: Syntax error, ',' expected
(7,13): error CS1003: Syntax error, ',' expected
(8,6): error CS1002: ; expected
(10,13): error CS1525: Invalid expression term '|'
(11,15): error CS1002: ; expected
(11,25): error CS1002: ; expected
(11,25): error CS1525: Invalid expression term '|'
(12,15): error CS1002: ; expected
(12,25): error CS1002: ; expected

This is basically because Which is a wrapper for Choice1 and Choice2, so we'd like to keep track of the actual entities rather than the wrapper.

#### Not Everything is Equal


The problem now is that despite us giving our entities ids, they still aren't equal, as we are still relying on structural equality.

In [11]:
type ContactId = ContactId of int
type PhoneNumber = PhoneNumber of int
type EmailAddress = EmailAddress of string

type Contact = {
    ContactId : ContactId
    PhoneNumber : PhoneNumber
    EmailAddress : EmailAddress
    }

//This is admittedly Object Oriented code but it's just to show off overriding 

//So let's define a couple of contacts:
let contactId = ContactId 1
let contact1 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber 1
    EmailAddress = EmailAddress "email"
    }

let contact2 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber 2
    EmailAddress = EmailAddress "email2"
    }

printfn "%b" (contact1 = contact2)

Error: (1,28): error CS1002: ; expected
(1,31): error CS1002: ; expected
(2,6): error CS1002: ; expected
(2,32): error CS1002: ; expected
(2,35): error CS1002: ; expected
(3,6): error CS1002: ; expected
(3,34): error CS1002: ; expected
(3,37): error CS1002: ; expected
(5,6): error CS1002: ; expected
(5,16): error CS1525: Invalid expression term '{'
(5,16): error CS1002: ; expected
(7,17): error CS1002: ; expected
(7,17): error CS1513: } expected
(8,18): error CS1002: ; expected
(8,18): error CS1513: } expected
(8,32): error CS1002: ; expected
(14,27): error CS1002: ; expected
(14,28): error CS1002: ; expected
(16,26): error CS1003: Syntax error, ',' expected
(17,31): error CS1003: Syntax error, ',' expected
(17,32): error CS1003: Syntax error, ',' expected
(18,33): error CS1003: Syntax error, ',' expected
(19,6): error CS1002: ; expected
(22,26): error CS1003: Syntax error, ',' expected
(23,31): error CS1003: Syntax error, ',' expected
(23,32): error CS1003: Syntax error, ',' expected


To combat that, we have to overwrite some stuff (specifically equals and hashcode):

In [12]:
[<CustomEquality; NoComparison>]
type EqualityContact = {
    ContactId : ContactId
    PhoneNumber : PhoneNumber
    EmailAddress : EmailAddress
    }
    with
    override this.Equals(obj) =
        match obj with 
        | :? EqualityContact as c -> this.ContactId = c.ContactId
        | _ -> false
    override this.GetHashCode() =
        hash this.ContactId 

//This is admittedly Object Oriented code but it's just to show off overriding 

//So let's define a couple of contacts:
let contactId = ContactId 1
let contact1 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber 1
    EmailAddress = EmailAddress "email"
    }

let contact2 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber 2
    EmailAddress = EmailAddress "email2"
    }

printfn "%b" (contact1 = contact2)

Error: (1,2): error CS1001: Identifier expected
(1,2): error CS1003: Syntax error, ',' expected
(1,3): error CS1003: Syntax error, ',' expected
(1,17): error CS1003: Syntax error, ',' expected
(1,19): error CS1003: Syntax error, ',' expected
(1,31): error CS1003: Syntax error, ',' expected
(3,15): error CS1003: Syntax error, ',' expected
(3,17): error CS1003: Syntax error, ',' expected
(3,26): error CS1003: Syntax error, ',' expected
(4,17): error CS1003: Syntax error, ',' expected
(4,19): error CS1003: Syntax error, ',' expected
(4,30): error CS1003: Syntax error, ',' expected
(5,18): error CS1003: Syntax error, ',' expected
(5,20): error CS1003: Syntax error, ',' expected
(6,6): error CS1003: Syntax error, ',' expected
(22,40): error CS1002: ; expected
(23,5): error CS7017: Member definition, statement, or end-of-file expected
(26,26): error CS1003: Syntax error, ',' expected
(27,31): error CS1003: Syntax error, ',' expected
(27,32): error CS1003: Syntax error, ',' expected
(28,33): 

Fun stuff. Also, a little bonus bit here, if we fancy not allowing equality testing, we can use [<NoEquality: NoComparison>] before the type. Useful if we want to be explicit about what is actually equal in an entity- we should comapre the id's not the the actual entity.

### Immutability
It's functional programming, so we want things to be immutable. For the Value Objects, that's easy. Just make a new one with the changed value.

Entities are slightly harder, but it's pretty much the same- copy the initial entity, change the field you want to change, and store the copy as a new entity (but do keep the id's the same- it might be a new entity decleration, but we're still working with the same thing.)


### Aggregates
Entities also pose problems if they're nested. Entities within Entities is known as an _aggregate_, with the top level being the _aggregate root_. If you're making changes to an entitty within the aggregae, really these changes have to be made at the aggregate root level. Say we have an Account entitiy, which within it has a customer entity, which within it has a customer name. If we change the name, we need to make a copy of the _Account_, not the customer name!


#### References

Let's look at a new example, with a customer making orders:

In [13]:
type Order = {
    OrderId : OrderId
    Customer: Customer
    OtherStuff: OtherStuff
}

//In this instance, Customer is an Entity. But what if we want to change Customer? We then have to change the order as well which due to immutability is a nuisance. It's probably better to reference the Customer instead:

type BetterOrder = {
    OrderId : OrderId
    CustomerId: CustomerId
    OtherStuff: OtherStuff
}

Error: (2,13): error CS1003: Syntax error, ',' expected
(2,15): error CS1003: Syntax error, ',' expected
(2,22): error CS1003: Syntax error, ',' expected
(3,13): error CS1003: Syntax error, ',' expected
(3,15): error CS1003: Syntax error, ',' expected
(3,23): error CS1003: Syntax error, ',' expected
(4,15): error CS1003: Syntax error, ',' expected
(4,17): error CS1003: Syntax error, ',' expected
(5,2): error CS1002: ; expected
(10,13): error CS1003: Syntax error, ',' expected
(10,15): error CS1003: Syntax error, ',' expected
(10,22): error CS1003: Syntax error, ',' expected
(11,15): error CS1003: Syntax error, ',' expected
(11,17): error CS1003: Syntax error, ',' expected
(11,27): error CS1003: Syntax error, ',' expected
(12,15): error CS1003: Syntax error, ',' expected
(12,17): error CS1003: Syntax error, ',' expected
(13,2): error CS1002: ; expected

BetterOrder is better, because then we can jsut look up Customer. If Customer is changed, we don't care! We don't use the Customer fields directly here. Customer and Order have become independent and distinct aggregates. They look after themselves and are only linked by their identifiers in the root aggregate.

A final note here, when retrieving aggregates always retrieve the whole thing. Then there's no chance of changing part of something but not the whole. If you think you only need part of an entity, maybe make a new entity for the stuff you need!

Aggregates will be everywhere in a domain, so it's a very good idea to define them first as part of the modelling process rather thann implementation

The idea of aggregates comes up a lot later on in the book, so I'd recommend reading the chapter. It covers a lot of this in more detail, especially as I ran out of time :/ 